In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import glob
import sys
import numpy
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
from math import ceil
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, AveragePooling1D, Conv1D, Activation, UpSampling1D, Reshape
from tcn import TCN, tcn_full_summary
from tcn import compiled_tcn
from mango.tuner import Tuner
from keras_flops import get_flops
from tqdm import tqdm
import common as com
import keras_model
import eval_functions_eembc
from sklearn import metrics
from hardware_utils import *
import pickle
import csv
import time
import itertools
import random

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6



In [2]:
NAS_EPOCHS = 50 #NAS epochs
device = "NUCLEO_L4R5ZI_P" #hardware name
dirpath='AD_Mbed_Prog/' #mbed program directory
model_name = "ADmodel"+device+"_"+".h5"
platform_connected = True #HIL or proxy
quantization = True #use quantization
os.system("mkdir -p trained_models/")

log_file_name = 'log_NAS_AD'+str(platform_connected)+"_"+device+'.csv' #log file for NAS
if os.path.exists(log_file_name):
    os.remove(log_file_name)
row_write = ['score', 'accuracy','SRAM','Flash','Latency',
             'nb_filters','kernel_size','dilations','nb_stacks','use_skip_connections']
with open(log_file_name, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(row_write)
if os.path.exists(log_file_name[0:-4]+'.p'):
    os.remove(log_file_name[0:-4]+'.p')

## Import Dataset

In [3]:
param = com.yaml_load()
os.makedirs(param["model_directory"], exist_ok=True)
dirs = com.select_dirs(param=param, mode=1)
idx = 0
target_dir = dirs[0]
files = com.file_list_generator(target_dir)
train_data = com.list_to_vector_array(files,
                                  msg="generate train_dataset",
                                  n_mels=param["feature"]["n_mels"],
                                  frames=param["feature"]["frames"],
                                  n_fft=param["feature"]["n_fft"],
                                  hop_length=param["feature"]["hop_length"],
                                  power=param["feature"]["power"])

2022-09-27 13:49:43,735 - INFO - load_directory <- development
2022-09-27 13:49:43,736 - INFO - target_dir : /home/nesl/swapnil/thin-bayes/Anomaly Detection Benchmark/l4r5zi_p_h/dev_data/ToyCar
2022-09-27 13:49:43,744 - INFO - train_file num : 4000
generate train_dataset: 100%|██████████████| 4000/4000 [00:34<00:00, 114.41it/s]


## Training and NAS

In [8]:
def objective_NN(nb_filters=6,kernel_size=32,dilations=[1,2,4,8,16,32,64,128],
                 nb_stacks=1, use_skip_connections = True, platform_connected = False, quantization=True):
    print(nb_filters,kernel_size,dilations,nb_stacks, use_skip_connections)
    training_flag = 0
    score = -5.0
    accuracy = -1.0
    ################################################################################
    i = Input((param["feature"]["n_mels"] * param["feature"]["frames"]))
    x = Reshape((1,param["feature"]["n_mels"] * param["feature"]["frames"]))(i)
    tcn_enc = TCN(nb_filters=nb_filters, kernel_size=kernel_size, nb_stacks=nb_stacks, dilations=dilations, 
                  padding='same', use_skip_connections=use_skip_connections, return_sequences=True,
                  kernel_initializer='glorot_normal', name='tcn-enc')(x)
    enc_flat = Conv1D(filters=ceil(nb_filters/4), kernel_size=ceil(kernel_size/3), activation='linear', padding='same')(tcn_enc)
    dec_upsample = Conv1D(filters=ceil(nb_filters/4), kernel_size=ceil(kernel_size/3), activation='linear', padding='same')(enc_flat)
    dec_reconstructed = TCN(nb_filters=nb_filters, kernel_size=kernel_size, nb_stacks=nb_stacks, dilations=dilations, 
                  padding='same', use_skip_connections=use_skip_connections, return_sequences=False,
                  kernel_initializer='glorot_normal', name='tcn-dec')(dec_upsample)
    o = Dense(param["feature"]["n_mels"] * param["feature"]["frames"], activation='linear')(dec_reconstructed)
    model = Model(inputs=[i], outputs=[o])
    ################################################################################
    model.compile(**param["fit"]["compile"])
    maxRAM, maxFlash = return_hardware_specs(device)
    if(platform_connected == False):
        Latency = get_flops(model, batch_size=1) #latency proxy
        RAM = get_model_memory_usage(batch_size=1, model=model) #SRAM proxy
        if(quantization==True):
            RAM = RAM/8.0
        Flash =  get_model_flash_usage(model,'g_model',quantization=quantization,train_data=train_data) #flash proxy
        if(RAM < maxRAM and Flash < maxFlash):
            training_flag = 1
        else:
            training_flag = 0
            score = -5.0
            accuracy = -1.0
    else:
        RAM, Flash, Latency, err_flag = platform_in_the_loop_controller(model,'g_model',
                                            device,train_data,dir_path=dirpath,quantization=quantization)
        if(RAM!=-1 and Flash!=-1 and Latency!=-1):
            training_flag = 1
        else:
            training_flag = 0
            score = -5.0
            accuracy = -1.0
            
    if(training_flag == 1):
        history = model.fit(train_data,
                            train_data,
                            epochs=param["fit"]["epochs"],
                            batch_size=param["fit"]["batch_size"],
                            shuffle=param["fit"]["shuffle"],
                            validation_split=param["fit"]["validation_split"],
                            verbose=param["fit"]["verbose"])
        accuracy = 1/history.history['val_loss'][-1]
        if(platform_connected == False):
            score = accuracy + 0.01*((RAM/maxRAM) + (Flash/maxFlash)) +  0.01*(Latency/1e6)
        else:
            score = accuracy + 0.01*((RAM/maxRAM) + (Flash/maxFlash)) +  0.01*(Latency/0.8)
    
    row_write = [score, accuracy,RAM,Flash,Latency,
             nb_filters,kernel_size,dilations,nb_stacks,use_skip_connections]
    print('Design choice:',row_write)
    with open(log_file_name, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(row_write)     
    
    return score

In [9]:
def save_res(data, file_name):
    pickle.dump( data, open( file_name, "wb" ) )
    
min_layer = 3
max_layer = 8
a_list = [1,2,4,8,16,32,64,128]
all_combinations = []
dil_list = []
for r in range(len(a_list) + 1):
    combinations_object = itertools.combinations(a_list, r)
    combinations_list = list(combinations_object)
    all_combinations += combinations_list
all_combinations = all_combinations[1:]
for item in all_combinations:
    if(len(item) >= min_layer and len(item) <= max_layer):
        dil_list.append(list(item))

param_dict = {
    'nb_filters': range(3,64),
    'nb_stacks': [1,2,3],
    'kernel_size': range(3,16),
    'use_skip_connections': [True, False],
    'dilations': dil_list
}

def objfunc(args_list):

    objective_evaluated = []
    
    start_time = time.time()
    
    for hyper_par in args_list:
        nb_filters = hyper_par['nb_filters']
        nb_stacks = hyper_par['nb_stacks']
        kernel_size = hyper_par['kernel_size']
        use_skip_connections = hyper_par['use_skip_connections']
        dilations = hyper_par['dilations']
            
        objective = objective_NN(nb_filters=nb_filters,kernel_size=kernel_size,dilations=dilations,
                 nb_stacks=nb_stacks, use_skip_connections = True, 
                                 platform_connected = platform_connected, quantization=quantization)
        objective_evaluated.append(objective)
        
        end_time = time.time()
        print('objective:', objective, ' time:',end_time-start_time)
        
    return objective_evaluated

In [ ]:
conf_Dict = dict()
conf_Dict['batch_size'] = 1 
conf_Dict['num_iteration'] = NAS_EPOCHS
conf_Dict['initial_random']= 5
tuner = Tuner(param_dict, objfunc,conf_Dict)
all_runs = []
results = tuner.maximize()
all_runs.append(results)
save_res(all_runs,log_file_name[0:-4]+'.p')

25 7 [1, 4, 16, 64] 2 True
INFO:tensorflow:Assets written to: /tmp/tmpaiy3mmlp/assets
INFO:tensorflow:Assets written to: /tmp/tmpr6t1grlm/assets


## Train the Best Model and Evaluate

In [ ]:
nb_filters = results['best_params']['nb_filters']
nb_stacks = results['best_params']['nb_stacks']
kernel_size = results['best_params']['kernel_size']
use_skip_connections = results['best_params']['use_skip_connections']
dilations = results['best_params']['dilations']

In [ ]:
i = Input((param["feature"]["n_mels"] * param["feature"]["frames"]))
x = Reshape((1,param["feature"]["n_mels"] * param["feature"]["frames"]))(i)
tcn_enc = TCN(nb_filters=nb_filters, kernel_size=kernel_size, nb_stacks=nb_stacks, dilations=dilations, 
              padding='same', use_skip_connections=use_skip_connections, return_sequences=True,
              kernel_initializer='glorot_normal', name='tcn-enc')(x)
enc_flat = Conv1D(filters=ceil(nb_filters/4), kernel_size=ceil(kernel_size/3), activation='linear', padding='same')(tcn_enc)
dec_upsample = Conv1D(filters=ceil(nb_filters/4), kernel_size=ceil(kernel_size/3), activation='linear', padding='same')(enc_flat)
dec_reconstructed = TCN(nb_filters=nb_filters, kernel_size=kernel_size, nb_stacks=nb_stacks, dilations=dilations, 
              padding='same', use_skip_connections=use_skip_connections, return_sequences=True,
              kernel_initializer='glorot_normal', name='tcn-dec')(dec_upsample)
o = Dense(param["feature"]["n_mels"] * param["feature"]["frames"], activation='linear')(dec_reconstructed)
model = Model(inputs=[i], outputs=[o])
################################################################################
model.compile(**param["fit"]["compile"])
history = model.fit(train_data,
                    train_data,
                    epochs=param["fit"]["epochs"],
                    batch_size=param["fit"]["batch_size"],
                    shuffle=param["fit"]["shuffle"],
                    validation_split=param["fit"]["validation_split"],
                    verbose=param["fit"]["verbose"])

In [ ]:
model.save("trained_models/" + model_name)

In [ ]:
machine_id_list = com.get_machine_id_list_for_test(target_dir)
csv_lines = []
csv_lines.append(["id", "AUC", "pAUC"])
performance = []
for id_str in machine_id_list:
    test_files, y_true = com.test_file_list_generator(target_dir, id_str, "development")
    anomaly_score_list = []
    y_pred = [0. for k in test_files]
    for file_idx, file_path in tqdm(enumerate(test_files), total=len(test_files)):
        data = com.file_to_vector_array(file_path,
                                        n_mels=param["feature"]["n_mels"],
                                        frames=param["feature"]["frames"],
                                        n_fft=param["feature"]["n_fft"],
                                        hop_length=param["feature"]["hop_length"],
                                        power=param["feature"]["power"])
        pred = model.predict(data)
        errors = numpy.mean(numpy.square(data - pred), axis=1)
        y_pred[file_idx] = numpy.mean(errors)
        anomaly_score_list.append([os.path.basename(file_path), y_pred[file_idx]])
        
    auc = metrics.roc_auc_score(y_true, y_pred)
    p_auc = metrics.roc_auc_score(y_true, y_pred, max_fpr=param["max_fpr"])
    csv_lines.append([id_str.split("_", 1)[1], auc, p_auc])
    performance.append([auc, p_auc])
    acc_eembc = eval_functions_eembc.calculate_ae_accuracy(y_pred, y_true)
    pr_acc_eembc = eval_functions_eembc.calculate_ae_pr_accuracy(y_pred, y_true)
    auc_eembc = eval_functions_eembc.calculate_ae_auc(y_pred, y_true, "dummy")

averaged_performance = numpy.mean(numpy.array(performance, dtype=float), axis=0)
csv_lines.append(["Average"] + list(averaged_performance))
csv_lines.append([])